In [1]:
import os
import json

for survey in os.listdir('human_resp'):
    if not survey.startswith('American_Trends_Panel'): continue
    with open('human_resp/' + survey + '/response_data_balanced_test_anno_1000.json', 'r') as f:
        data = json.load(f)
    res = {}
    for entry in data:
        user = entry['author_id']
        q_key = entry['conversation_id']
        if user not in res:
            res[user] = [q_key]
        else:
            res[user].append(q_key)
    # with open('human_resp/' + survey + '/user_test_q_key_mapping.json', 'w') as f:
    #     json.dump(res, f, indent=4)

In [3]:
import os
os.chdir("/home/ubuntu/code/pump_post_midterm/pump")

import pandas as pd
from src.utils import get_file_from_s3

file_key = "American_Trends_Panel_W34"
survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{file_key}/info.csv"))
resp_df = pd.read_csv(get_file_from_s3(f"human_resp/{file_key}/responses.csv"))
meta_df = pd.read_csv(get_file_from_s3(f"human_resp/{file_key}/metadata.csv"))
question_key_mapping = {row['key']: {'question': row['question'], 'references': row['references']} for _, row in survey_df.iterrows()}
meta_keys = list(meta_df['key'])
q_keys = list(survey_df['key'])

with open(f'experiment/data/human_resp/American_Trends_Panel_W34/user_test_q_key_mapping.json', 'r') as f:
    user_test_q_key_mapping = json.load(f)
user_train_q_key_mapping = {}
for user in user_test_q_key_mapping:
    user_train_q_key_mapping[user] = [_ for _ in q_keys if _ not in user_test_q_key_mapping[user]]
for user in range(len(resp_df)):
    user = str(user)
    if user not in user_test_q_key_mapping:
        user_train_q_key_mapping[user] = q_keys
# set up dfs
test_user_idx = [int(_) for _ in user_test_q_key_mapping.keys()]
train_user_idx = [_ for _ in range(len(resp_df)) if _ not in test_user_idx]
test_resp_df = resp_df.iloc[test_user_idx]
train_resp_df = resp_df.iloc[train_user_idx]

In [19]:
with open(f'experiment/data/human_resp/{file_key}/user_test_q_key_mapping.json', 'r') as f:
    user_test_q_key_mapping = json.load(f)
user_train_q_key_mapping = {}
for user in user_test_q_key_mapping:
    user_train_q_key_mapping[user] = [_ for _ in q_keys if _ not in user_test_q_key_mapping[user]]
for user in range(len(resp_df)):
    user = str(user)
    if user not in user_test_q_key_mapping:
        user_train_q_key_mapping[user] = q_keys
# set up dfs
test_user_idx = [int(_) for _ in user_test_q_key_mapping.keys()]
train_user_idx = [_ for _ in range(len(resp_df)) if _ not in test_user_idx]
test_resp_df = resp_df.iloc[test_user_idx]
train_resp_df = resp_df.iloc[train_user_idx]

In [21]:
test_resp_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,QKEY,Device_Type_W34,LANGUAGE_W34,FORM_W34,SCI1_W34,SCI2A_W34,...,F_REG_FINAL,POLIDEOLOGY,F_INTUSER_FINAL,F_VOLSUM_FINAL,F_GENERATIONS_FINAL,F_RELCOM3CAT_FINAL,PRAY_FINAL,RELIMP_FINAL,WEIGHT_W34,RACE
947,947,947,947,947,6.741110e+05,Desktop,English,Form 2,Easier,Mostly positive,...,You are ABSOLUTELY CERTAIN that you are regist...,Very liberal,Internet User,No,Xer,Low,Never,Not at all important,1.199404,White
698,698,698,698,698,6.264220e+05,Mobile phone,English,Form 2,Easier,Mostly positive,...,You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Internet User,Yes,Millennial,Medium,Once a week,Very important,0.099012,Black
141,141,141,141,141,1.511300e+05,Desktop,English,Form 1,Easier,Mostly positive,...,You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Internet User,Yes,Boomer,High,Once a day,Very important,0.157426,Black
1624,1624,1624,1624,1624,2.015016e+11,Tablet,English,Form 2,Easier,Mostly negative,...,You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Internet User,Yes,Boomer,High,Several times a day,Very important,0.140072,Black
2163,2163,2163,2163,2163,2.017016e+11,Mobile phone,English,Form 2,More difficult,Mostly negative,...,You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Internet User,No,Xer,Medium,A few times a week,Very important,0.759779,Black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2235,2235,2235,2235,2.017016e+11,Mobile phone,English,Form 2,Easier,Mostly positive,...,You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Internet User,Yes,Millennial,High,Several times a day,Very important,1.560725,White
876,876,876,876,876,6.621080e+05,Mobile phone,English,Form 2,Easier,Mostly positive,...,You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Internet User,Yes,Xer,Medium,A few times a week,Somewhat important,0.091151,Black
1539,1539,1539,1539,1539,2.015016e+11,Desktop,English,Form 1,Easier,Mostly positive,...,You are NOT registered to vote at your current...,Liberal,Internet User,No,Boomer,Low,Seldom,Not at all important,0.332598,Hispanic
106,106,106,106,106,1.351790e+05,Desktop,English,Form 1,Easier,Mostly positive,...,You are ABSOLUTELY CERTAIN that you are regist...,Very liberal,Internet User,No,Boomer,NaN,NaN,Not too important,0.455172,White


In [7]:
seen = []
for idx, row in test_resp_df.iterrows():
    seen.append(idx)
    if idx == '697': break

In [25]:
with open("opinions_qa/persona_val/American_Trends_Panel_W34/date0904_personas_full_personadb.json") as f:
    personas = json.load(f)

In [27]:
missing = []
for user, row in test_resp_df.iterrows():
    if str(user) not in personas:
        missing.append(user)

In [28]:
len(missing)

0

In [31]:
"697" in user_test_q_key_mapping

False

In [34]:
from tqdm import tqdm 

for user_idx, row in tqdm(test_resp_df.iterrows(), total=len(test_resp_df)):
    if user_idx == 697:
        print(user_idx)

100%|██████████| 848/848 [00:00<00:00, 29642.71it/s]
